In [4]:
import matplotlib
import re
import os
import sys

In [5]:
capture_src = 'captures'
capture_dst = 'captures_split'
processed = 'processed_log.txt'

In [ ]:
class packet:
    packet_string = ""
    ip_length = 0
    tcp_length = 0
    receive_time = 0

In [13]:
def get_packets_in_file(capture_file):
    '''
    packet_capture_file = open(capture_file,'r')
    packet_capture_records = packet_capture_file.read()
    packet_capture_file.close()
    packets = re.findall(".*IP.*length.*Flags.*length.*\n",packet_capture_records,re.MULTILINE)
    '''
    
    line_one = ''
    line_two = ''
    packets = list()
    with open(capture_file,'r') as src:
        for line in src:
            line_one = line_two
            line_two = line
            line_pair = line_one + line_two
            
            packet_match = re.match(r'.*IP.*length.*\r?\n.*Flags.*length.*',line_pair,re.MULTILINE)
            
            if(packet_match):
                print(line_pair)
                raise KeyboardInterrupt
            
    
    return packets

In [14]:
try:
    for x in os.walk(capture_src): #each os.walk element is: dirpath, subdir-names, dir-filenames
        outdir = x[0].replace(capture_src,capture_dst)
        try:
            os.mkdir(outdir)
        except FileExistsError:
            pass #We don't care if the directory already exists
        for capture in x[2]:
            if capture.endswith(".cap"):
                base_file = capture[:-4]
                date_file = base_file + '.date'
                capture_packets = get_packets_in_file(x[0]+"\\"+capture)
                
                for packet in capture_packets:
                    print(packet)
                    raise KeyboardInterrupt

                        
except KeyboardInterrupt:
    pass
            
    #print(outdir)

2018-09-30 00:27:33.133058 IP (tos 0x10, ttl 64, id 40973, offset 0, flags [DF], proto TCP (6), length 104)
    ShadowSense.shadowdomain.ssh > Shadowscout.shadowdomain.52303: Flags [P.], cksum 0xb439 (correct), seq 2878089682:2878089746, ack 1345784929, win 513, length 64



In [ ]:
#Isolate the outputs into <src_name>.connectionNumber.cap
##Then plot the connections onto a single graph.
###The dominant connection in a packet capture is the one you keep